In [1]:
import numpy as np
import pandas as pd
from datetime import datetime,date
import time

import cx_Oracle
import csv
import math
import pandas as pd
import numpy as np

#加减年月
import dateutil.relativedelta
# from datetime import datetime
# from datetime import date

import matplotlib.pyplot as plt
import os

from datetime import *
dns_tns=cx_Oracle.makedsn('172.16.50.233',1521,service_name='dfcfstb')
conn = cx_Oracle.Connection(user='cjchenjm5',password='Jm0228_Cjsc!',dsn=dns_tns,encoding='UTF-16')

In [39]:
# 取每月最后一个交易日
def get_last_tradate(date0,date1):
    date0 = date0.strftime('%Y%m%d')
    date1 = date1.strftime('%Y%m%d')   
    sql = f'''
                select 交易日期,month
                from
                (
                    select 交易日期,month,row_number() over(partition by month order by 交易日期 desc) as rank
                    from
                    (
                        SELECT F1_1010 交易日期,substr(F1_1010,1,6) month
                        FROM wind.TB_OBJECT_1010
                        where F1_1010 <= '{date1}'
                        and F1_1010 >= '{date0}'
                    )
                )t
                where t.rank = 1
            '''
    data = pd.read_sql(sql, conn)
    return data

time_list = get_last_tradate(datetime(2010,12,31),date.today())
time_list['交易日期'] = pd.to_datetime(time_list['交易日期'])
space_month=time_list

e:\anocanda\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [ ]:
# 原版本
def get_cap_turnover(date0):
    #d0和d1相差一年
    #输入调仓日，生成其一年前的日期,datetime格式
    #时间加减（减一年）
    d0 = date0 - pd.DateOffset(years=1)
    date1 = d0.strftime('%Y%m%d')
    sql = f'''
                    select F16_1090,F7_0002,F17_1090 上市时间,F18_1090 摘牌日期,avg_cap 日均总市值,avg_to 日均成交额
                    from(
                        select F16_1090,F7_0002,F17_1090,F18_1090,avg(总市值) avg_cap,avg(成交额) avg_to
                        from(
                            SELECT
                                F16_1090,
                                F7_0002,
                                F9_5004/10000 总市值,
                                F11_1120/100000 成交额,
                                F17_1090,F18_1090
                            FROM
                                wind.TB_OBJECT_5004
                            LEFT JOIN wind.TB_OBJECT_1090 on F2_1090 = F1_5004
                            left join wind.TB_OBJECT_0002 ON F27_1090 = F4_0002
                            left join wind.TB_OBJECT_1120 on F1_1120 = F2_1090
                            WHERE F2_5004 <= '{date0}'and F2_5004 >= '{date1}'
                            and F2_1120 <= '{date0}'and F2_1120 >= '{date1}'
                            AND F4_1090 = 'A'
                            and F17_1090 <= '{date0}'
                            and F2_1120 = F2_5004
                        )
                        group by F16_1090,F7_0002,F17_1090,F18_1090
                    )
            '''
    data = pd.read_sql(sql, conn)
    data['证券代码'] = data['F16_1090'].str.cat(data['F7_0002'],sep = '.')
    data['日期'] = date0
    data = data[['日期','证券代码','上市时间','摘牌日期','日均总市值','日均成交额']]
    data['cap_pct'] = data.groupby('日期').rank(pct= True)['日均总市值']
    data['to_pct'] = data.groupby('日期').rank(pct= True)['日均成交额']
    data = data.loc[(data['cap_pct']>0.2)&(data['to_pct']>0.2)]
    data.sort_values(by=['证券代码'],inplace=True)
    return data

#剔除市值和流动性后20%
pool = pd.DataFrame()
for i in range(len(space_month)):
    date = space_month['交易日期'][i]#datetime
    temp = get_cap_turnover(date)
    pool=pool.append(temp)


In [ ]:
pool.to_excel('D:/实习/Python参考资料/'+'pool.xlsx')
pool

In [ ]:
# 剔除条件 
def get_cap_turnover(date0):
    #d0和d1相差一年
    #输入调仓日，生成其一年前的日期,datetime格式
    #时间加减（减一年）
    d0 = date0 - pd.DateOffset(years=1)
    date1 = d0.strftime('%Y%m%d')
    sql = f'''
                    select F16_1090,F7_0002,F17_1090 上市时间,F18_1090 摘牌日期,avg_cap 日均总市值,avg_to 日均成交额, avg_not_roe 扣非ROE均值, F9_5004 总市值            
                    from(
                        select F16_1090,F7_0002,F17_1090,F18_1090,avg(总市值) avg_cap,avg(成交额) avg_to, avg(扣非ROE) avg_not_roe, F9_5004
                        from(
                            SELECT
                                F16_1090,
                                F7_0002,
                                F9_5004/10000 总市值,
                                F11_1120/100000 成交额,
                                F61_1683 扣非ROE,
                                F9_5004,
                                
                                F17_1090,F18_1090
                                
                            FROM
                                wind.TB_OBJECT_5004
                            LEFT JOIN wind.TB_OBJECT_1090 on F2_1090 = F1_5004
                            left join wind.TB_OBJECT_0002 ON F27_1090 = F4_0002
                            left join wind.TB_OBJECT_1120 on F1_1120 = F2_1090
                            left join wind.TB_OBJECT_1683 on F1_1683 = F16_1090
                            WHERE F2_5004 <= '{date0}'and F2_5004 >= '{date1}'
                            and F2_1120 <= '{date0}'and F2_1120 >= '{date1}'
                            AND F4_1090 = 'A'  
                            and F17_1090 <= '{date0}'
                            and F2_1120 = F2_5004
                        )
                        group by F16_1090,F7_0002,F17_1090,F18_1090,F9_5004
                    )
            '''
    data = pd.read_sql(sql, conn)
    data['证券代码'] = data['F16_1090'].str.cat(data['F7_0002'],sep = '.')
    data['日期'] = date0
    data = data[['日期','证券代码','上市时间','摘牌日期','日均总市值','日均成交额']]
    data['cap_pct'] = data.groupby('日期').rank(pct= True)['日均总市值']
    data['to_pct'] = data.groupby('日期').rank(pct= True)['日均成交额']
    # data = data.loc[(data['cap_pct']>0.2)&(data['to_pct']>0.2)]
    # data = data.loc[data['to_pct']>0.2] # 过去一年日均成交额
    data.sort_values(by=['证券代码'],inplace=True)
    return data

#剔除市值和流动性后20%
pool = pd.DataFrame()
for i in range(len(space_month)):
    date = space_month['交易日期'][i]#datetime
    temp = get_cap_turnover(date)
    pool=pool.append(temp)

In [ ]:
# 测试版本 剔除条件
date0=datetime(2020,12,1)
d0 = date0 - pd.DateOffset(years=1)
date1 = d0.strftime('%Y%m%d')
date0 = date0.strftime('%Y%m%d')
sql= f'''
            select 
                F16_1090,F7_0002,avg(F53_5034) 扣非ROE, F31_1683 每股股利
            from(
                select F16_1090,F7_0002,F53_5034,F31_1683
                FROM
                    wind.TB_OBJECT_5034
                LEFT JOIN wind.TB_OBJECT_1090 on F16_1090 = F1_5034
                left join wind.TB_OBJECT_0002 ON F27_1090 = F4_0002
                left join wind.TB_OBJECT_1120 on F1_1120 = F2_1090
                left join wind.TB_OBJECT_5004 on F1_5004 = F2_1090
                left join wind.TB_OBJECT_1683 on F1_1683 = F16_1090
                WHERE F3_5034 <= '{date0}'and F3_5034 >= '{date1}'
                and F2_1120 <= '{date0}'and F2_1120 >= '{date1}'
                AND F4_1090 = 'A'  
                and F17_1090 <= '{date0}'
                and F53_5034 is not null
                )
               group by F16_1090,F7_0002, F31_1683
            
    '''
data = pd.read_sql(sql, conn)

In [17]:
# 净利润
date0=datetime(2020,12,1)
d0 = date0 - pd.DateOffset(years=1)
date1 = d0.strftime('%Y%m%d')
date0 = date0.strftime('%Y%m%d')
sql=f'''
        select
            代码,日期,净利润,类型, F96, rank
        from(
            SELECT
                F16_1090 代码,
                F2_1854 日期,
                F3_1854 公告日期,
                F4_1854 类型,
                F61_1854 净利润,
                F96_1854 F96,
                row_number() over(partition by F2_1854,F16_1090 order by F96_1854 desc) as rank
            FROM
                wind.TB_OBJECT_1854
            LEFT JOIN wind.TB_OBJECT_1090 on Ob_revisions_1090 = F1_1854
            WHERE 
                F2_1854 <= '{date0}' and F2_1854 >= '{date1}' and F96_1854<= '{date0}' 
                AND F4_1090 = 'A' AND F4_1854 not like '%单季度%' and F2_1854 like '%1231' 
                AND F4_1854 like '%合并%') tt
            where 
            tt.rank=1
            
'''
data=pd.read_sql(sql,conn)

In [27]:
# 日均成交额
date0=datetime(2020,12,31)
d0 = date0 - pd.DateOffset(years=1)
date1 = d0.strftime('%Y%m%d')
date0 = date0.strftime('%Y%m%d')
sql=f'''
        select 
                代码, 日均成交量, 交易所后缀
        from(
             select
                    F16_1090 代码,
                    avg(F11_1120/100000) 日均成交量,   
                    F7_0002 交易所后缀
                from
                        wind.TB_OBJECT_1120
                left join wind.TB_OBJECT_1090 on F2_1090 = F1_1120
                left join wind.TB_OBJECT_0002 on F27_1090 = F4_0002
                where
                        F2_1120 <= '{date0}' and F2_1120 >= '{date1}'
                        AND F4_1090 = 'A'  and F17_1090 <= '{date0}'
                group by F16_1090, F7_0002
        )
                
'''
data_turnover=pd.read_sql(sql,conn)

e:\anocanda\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [30]:
# 剔除后20%
data_turnover['日期']=date0
data_turnover['turnover_day']=data_turnover.groupby('日期').rank(pct=True)['日均成交量']
data_turnover=data_turnover.loc[data_turnover['turnover_day']>0.2]

In [2]:
# 扣非ROE
date0=datetime(2020,12,31)
d0 = date0 - pd.DateOffset(years=2)
date1 = d0.strftime('%Y%m%d')
date0 = date0.strftime('%Y%m%d')

sql=f'''
    select
            代码, 日期, 净资产收益率_扣非, 交易所后缀
    from 
        (
        select 
                F16_1090 代码,
                F3_5034 日期,
                F53_5034 净资产收益率_扣非,
                F7_0002 交易所后缀, 
                F17_1090 上市时间,
                
                row_number() over(partition by F16_1090, F3_5034 order by F3_5034 desc) as rank
        from
                wind.TB_OBJECT_5034
        left join wind.TB_OBJECT_1090 on Ob_revisions_1090 = F1_5034
        left join wind.TB_OBJECT_0002 on F27_1090 = F4_0002
        
        where
                F3_5034 <= '{date0}' and F3_5034 >= '{date1}' and F17_1090 <= '{date0}'
                AND F4_1090 = 'A' and F3_5034 like '%1231'
        )
'''
data=pd.read_sql(sql,conn)


e:\anocanda\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [3]:
# 扣非roe均值
data1=data.copy()
data2=data1.groupby('代码').mean()['净资产收益率_扣非']

In [4]:
data3=data1[data1['日期']==date0].drop('净资产收益率_扣非',axis=1)
data4=pd.DataFrame(data2)
data4=data4.reset_index()
data4=data4.rename(columns={'净资产收益率_扣非':'扣非roe均值'})
data5=pd.merge(data3,data4,on='代码',how='outer') # 扣非ROE均值
data5=data5.sort_values('代码')

In [34]:
# 扣非roe标准差
data1s=data.copy()
data2s=data1s.groupby('代码').std()['净资产收益率_扣非']
data3s=data1s[data1s['日期']==date0].drop('净资产收益率_扣非',axis=1)
data4s=pd.DataFrame(data2s)
data4s=data4s.reset_index()
data4s=data4s.rename(columns={'净资产收益率_扣非':'扣非roe标准差'})
data5s=pd.merge(data3s,data4s,on='代码',how='outer') # 扣非ROE标准差
data5s=data5s.sort_values('代码')
# 均值-标准差
data5s['mean_std']=data5['扣非roe均值']-data5s['扣非roe标准差']
data_roe=data5s.copy()

In [35]:
# 连接成交量和roe
data_roe=pd.merge(data_turnover,data_roe,on='代码',how='left')

In [37]:
# 剔除后30%
data_roe=data_roe.drop(data_roe[np.isnan(data_roe['mean_std'])].index) # 去除na值
data_roe['roe_rank']=data_roe.groupby('日期_x').rank(pct=True)['mean_std']
data_roe=data_roe.loc[data_roe['roe_rank']>0.3]

In [18]:
# 分红
date0=datetime(2020,12,31)
d0 = date0 - pd.DateOffset(years=2)
date1 = d0.strftime('%Y%m%d')
date0 = date0.strftime('%Y%m%d')
sql=f'''
        select 
            代码, 分红年度,  派息比例分子,交易所后缀, rank
        from 
            (
            select 
                    F16_1090 代码,
                    F24_1093 分红年度,
                    F9_1093 派息比例分子,
                    F7_0002 交易所后缀,
                    F26_1093 除权除息日,
                    row_number() over(partition by F16_1090 order by F24_1093 desc) as rank
                    
            from
                wind.TB_OBJECT_1093
            left join wind.TB_OBJECT_1090 on F2_1090 = F1_1093
            left join wind.TB_OBJECT_0002 on F27_1090 = F4_0002
            where 
                F24_1093 <= '{date0}' 
                and F24_1093 >= '{date1}' and F4_1090 = 'A' and F24_1093 like '%1231'
            )

'''
data_divid=pd.read_sql(sql,conn)

e:\anocanda\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [19]:
data_divid1=data_divid.copy()
data_divid2=data_divid1.drop(data_divid1[np.isnan(data_divid1['派息比例分子'])].index) # 去除没有分红的年份
data_divid3=data_divid2.groupby('代码').filter(lambda x: len(x)==3) # 连续三年分红的股票
data_divid4=data_divid3[data_divid3['分红年度']==date0]
data_dividend=data_divid4.rename(columns={'分红年度':'日期'}) # 连续三年分红

In [38]:
# 组合dividend和roe
data_merge=pd.merge(data_dividend,data_roe,on='代码')